In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("transfusion_5%.csv")

In [3]:
df

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.0,50.0,12500.0,98.0,1.0
1,NaN,13.0,3250.0,NaN,1.0
2,1.0,16.0,4000.0,35.0,1.0
3,2.0,NaN,5000.0,45.0,1.0
4,1.0,24.0,6000.0,77.0,0.0
...,...,...,...,...,...
743,NaN,2.0,500.0,38.0,0.0
744,21.0,2.0,500.0,52.0,0.0
745,23.0,NaN,750.0,62.0,0.0
746,39.0,1.0,250.0,39.0,0.0


In [4]:
df.dtypes

Recency (months)                              float64
Frequency (times)                             float64
Monetary (c.c. blood)                         float64
Time (months)                                 float64
whether he/she donated blood in March 2007    float64
dtype: object

In [5]:
df.isnull().sum()

Recency (months)                              46
Frequency (times)                             46
Monetary (c.c. blood)                         55
Time (months)                                 40
whether he/she donated blood in March 2007     0
dtype: int64

In [6]:
df.isnull().sum().sum()

187

In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Recency (months)                              6.149733
Frequency (times)                             6.149733
Monetary (c.c. blood)                         7.352941
Time (months)                                 5.347594
whether he/she donated blood in March 2007    0.000000
dtype: float64

In [8]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 3740
nilai hilang : 187
persentase : 5.0


In [9]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",euc_dist[col])

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [10]:
from collections import Counter

def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count <= 2 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None    

def Final(df,col,k):
    terdekat = []
    mean = 0
    minimum = None
    maximum = None
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        if (minimum is None or maximum is None):
            minimum = y
            maximum = y
        elif(y<minimum):
            minimum = y
        elif(y>maximum):
            maximum = y
        mean += y
        euc_val.append(y)
    mean /= len(terdekat)
    euc_val.sort()
    print(euc_val)
    
    fuzzy = hitung(euc_val,9)
    print("min : ",minimum)
    print("max : ",maximum)
    print("mean : ",mean)
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,fuzzy_triangular(y,minimum,mean,maximum)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    
    if tertimbang == 0:
        tertimbang = dummy.iloc[index-1, col]
    
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [11]:
def fuzzy_triangular(val, minimum, mean, maximum):
   if val <= minimum or val >= maximum: 
        return 0.0
   elif minimum < val <= mean:
        return float((val - minimum)/(mean - minimum)) 
   elif mean < val < maximum:
        return float((maximum - val)/(maximum - mean))

In [12]:
data = df.iloc[:, :-1]
df2 = df.copy()
dummy = df2.fillna(0)
dummy

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.0,50.0,12500.0,98.0,1.0
1,0.0,13.0,3250.0,0.0,1.0
2,1.0,16.0,4000.0,35.0,1.0
3,2.0,0.0,5000.0,45.0,1.0
4,1.0,24.0,6000.0,77.0,0.0
...,...,...,...,...,...
743,0.0,2.0,500.0,38.0,0.0
744,21.0,2.0,500.0,52.0,0.0
745,23.0,0.0,750.0,62.0,0.0
746,39.0,1.0,250.0,39.0,0.0


In [13]:
euc_dist = {}

In [14]:
for i in range(df2.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,9)
dummy

Kolom : 1
[1.0, 2.0, 2.0, 2.0, 2.0, 4.0, 12.0, 14.0, 21.0]
min :  1.0
max :  21.0
mean :  6.666666666666667
fuzzy :  [(12.0, 0.627906976744186), (2.0, 0.1764705882352941), (4.0, 0.5294117647058824), (1.0, 0.0), (2.0, 0.1764705882352941), (14.0, 0.48837209302325585), (21.0, 0.0), (2.0, 0.1764705882352941), (2.0, 0.1764705882352941)]
fuzzy :  None
pembilang :  17.901504787961695
penyebut :  2.3515731874145005
rata-rata tertimbang :  7.612565445026178
 
[2.0, 2.0, 2.0, 2.0, 2.0, 4.0, 4.0, 14.0, 14.0]
min :  2.0
max :  14.0
mean :  5.111111111111111
fuzzy :  [(2.0, 0.0), (2.0, 0.0), (2.0, 0.0), (2.0, 0.0), (4.0, 0.6428571428571429), (14.0, 0.0), (14.0, 0.0), (4.0, 0.6428571428571429), (2.0, 0.0)]
fuzzy :  None
pembilang :  5.142857142857143
penyebut :  1.2857142857142858
rata-rata tertimbang :  4.0
 
[1.0, 2.0, 2.0, 4.0, 4.0, 7.0, 8.0, 12.0, 23.0]
min :  1.0
max :  23.0
mean :  7.0
fuzzy :  [(23.0, 0.0), (2.0, 0.16666666666666666), (8.0, 0.9375), (12.0, 0.6875), (7.0, 1.0), (2.0, 0.1666666

[0.0, 0.0, 11.0, 16.0, 21.0, 21.0, 21.0, 23.0, 23.0]
min :  0.0
max :  23.0
mean :  15.11111111111111
fuzzy :  [(0.0, 0.0), (21.0, 0.2535211267605634), (21.0, 0.2535211267605634), (11.0, 0.7279411764705882), (23.0, 0.0), (0.0, 0.0), (16.0, 0.8873239436619718), (21.0, 0.2535211267605634), (23.0, 0.0)]
fuzzy :  None
pembilang :  38.17636702568351
penyebut :  2.3758285004142503
rata-rata tertimbang :  16.068654374264415
 
[0.0, 4.0, 4.0, 4.0, 8.0, 11.0, 14.0, 14.0, 23.0]
min :  0.0
max :  23.0
mean :  9.11111111111111
fuzzy :  [(0.0, 0.0), (14.0, 0.648), (23.0, 0.0), (4.0, 0.43902439024390244), (4.0, 0.43902439024390244), (14.0, 0.648), (4.0, 0.43902439024390244), (8.0, 0.8780487804878049), (11.0, 0.864)]
fuzzy :  None
pembilang :  39.94068292682927
penyebut :  4.3551219512195125
rata-rata tertimbang :  9.170967741935483
 
[0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 14.0]
min :  0.0
max :  14.0
mean :  3.111111111111111
fuzzy :  [(0.0, 0.0), (2.0, 0.6428571428571428), (14.0, 0.0), (2.0, 0.64

[0.0, 0.0, 2.0, 9.0, 14.0, 14.0, 21.0, 21.0, 21.0]
min :  0.0
max :  21.0
mean :  11.333333333333334
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (2.0, 0.1764705882352941), (14.0, 0.7241379310344828), (14.0, 0.7241379310344828), (21.0, 0.0), (21.0, 0.0), (21.0, 0.0), (9.0, 0.7941176470588235)]
fuzzy :  None
pembilang :  27.775862068965516
penyebut :  2.418864097363083
rata-rata tertimbang :  11.483018867924528
 
[0.0, 2.0, 2.0, 4.0, 4.0, 4.0, 14.0, 23.0, 25.0]
min :  0.0
max :  25.0
mean :  8.666666666666666
fuzzy :  [(0.0, 0.0), (2.0, 0.23076923076923078), (4.0, 0.46153846153846156), (25.0, 0.0), (2.0, 0.23076923076923078), (14.0, 0.6734693877551019), (23.0, 0.12244897959183672), (4.0, 0.46153846153846156), (4.0, 0.46153846153846156)]
fuzzy :  None
pembilang :  18.706436420722135
penyebut :  2.642072213500785
rata-rata tertimbang :  7.080213903743315
 
[0.0, 2.0, 9.0, 11.483018867924528, 14.0, 14.0, 21.0, 21.0, 21.0]
min :  0.0
max :  21.0
mean :  12.609224318658281
fuzzy :  [(0.0, 0.0), (2.0, 0

[8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0]
median :  5
min :  8.0
max :  8.0
mean :  8.0
fuzzy :  [(8.0, 8.0), (8.0, 8.0), (8.0, 8.0), (8.0, 8.0), (8.0, 8.0), (8.0, 8.0), (8.0, 8.0), (8.0, 8.0), (8.0, 8.0)]
fuzzy :  8.0
pembilang :  576.0
penyebut :  72.0
rata-rata tertimbang :  8.0
 
[0.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]
median :  5
min :  0.0
max :  3.0
mean :  2.6666666666666665
fuzzy :  [(0.0, 3.0), (3.0, 3.0), (3.0, 3.0), (3.0, 3.0), (3.0, 3.0), (3.0, 3.0), (3.0, 3.0), (3.0, 3.0), (3.0, 3.0)]
fuzzy :  3.0
pembilang :  72.0
penyebut :  27.0
rata-rata tertimbang :  3.0
 
[0.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0]
median :  5
min :  0.0
max :  7.0
mean :  6.222222222222222
fuzzy :  [(0.0, 7.0), (7.0, 7.0), (7.0, 7.0), (7.0, 7.0), (7.0, 7.0), (7.0, 7.0), (7.0, 7.0), (7.0, 7.0), (7.0, 7.0)]
fuzzy :  7.0
pembilang :  392.0
penyebut :  63.0
rata-rata tertimbang :  7.0
 
[0.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]
median :  5
min :  0.0
max :  3.0
mean :  2.666666666666666

[1750.0, 2000.0, 2000.0, 2250.0, 2250.0, 2500.0, 2500.0, 2750.0, 2750.0]
min :  1750.0
max :  2750.0
mean :  2305.5555555555557
fuzzy :  [(2750.0, 0.0), (2000.0, 0.4499999999999999), (2500.0, 0.5625000000000001), (2500.0, 0.5625000000000001), (2250.0, 0.8999999999999998), (2250.0, 0.8999999999999998), (2750.0, 0.0), (1750.0, 0.0), (2000.0, 0.4499999999999999)]
fuzzy :  None
pembilang :  8662.5
penyebut :  3.8249999999999997
rata-rata tertimbang :  2264.7058823529414
 
[1250.0, 1250.0, 1500.0, 1750.0, 1750.0, 1750.0, 2000.0, 2000.0, 2000.0]
min :  1250.0
max :  2000.0
mean :  1694.4444444444443
fuzzy :  [(2000.0, 0.0), (2000.0, 0.0), (2000.0, 0.0), (1750.0, 0.8181818181818179), (1750.0, 0.8181818181818179), (1750.0, 0.8181818181818179), (1250.0, 0.0), (1500.0, 0.5625000000000001), (1250.0, 0.0)]
fuzzy :  None
pembilang :  5139.204545454544
penyebut :  3.0170454545454537
rata-rata tertimbang :  1703.3898305084747
 
[0.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1250.0, 1250.0, 1500.0]
min

...

[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.7777777777777778
fuzzy :  [(0.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  7.0
penyebut :  9.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.4444444444444444
fuzzy :  [(0.0, 0.0), (1.0, 0.0), (1.0, 0.0), (1.0, 0.0), (1.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.8888888888888888
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  8.0
penyebut :  9.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.6666666

[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.7777777777777778
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  7.0
penyebut :  9.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.7777777777777778
fuzzy :  [(0.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  7.0
penyebut :  9.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.7777777777777778
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  7.0
penyebut :  9.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.3333333

[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.8888888888888888
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  8.0
penyebut :  9.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.7777777777777778
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  7.0
penyebut :  9.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.7777777777777778
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  7.0
penyebut :  9.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.8888888

[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.8888888888888888
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  8.0
penyebut :  9.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.8888888888888888
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  8.0
penyebut :  9.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.7777777777777778
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 1.0)]
fuzzy :  1.0
pembilang :  7.0
penyebut :  9.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.8888888

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.000000,50.000000,12500.0,98.000000,1.0
1,7.612565,13.000000,3250.0,49.746725,1.0
2,1.000000,16.000000,4000.0,35.000000,1.0
3,2.000000,20.619469,5000.0,45.000000,1.0
4,1.000000,24.000000,6000.0,77.000000,1.0
...,...,...,...,...,...
743,11.280374,2.000000,500.0,38.000000,1.0
744,21.000000,2.000000,500.0,52.000000,1.0
745,23.000000,3.000000,750.0,62.000000,1.0
746,39.000000,1.000000,250.0,39.000000,1.0


In [15]:
dummy.describe()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
count,748.000000,748.000000,748.000000,748.000000,748.0
mean,9.571488,5.516871,1363.494581,33.834128,1.0
std,7.978559,5.840487,1443.157498,23.838226,0.0
min,1.000000,1.000000,250.000000,2.000000,1.0
25%,3.919170,2.000000,500.000000,16.000000,1.0
50%,8.000000,4.000000,1000.000000,28.000000,1.0
75%,14.000000,7.000000,1750.000000,48.000000,1.0
max,74.000000,50.000000,12500.000000,98.000000,1.0


# Klasifikasi

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [34]:
x = dummy.values
y = df.iloc[:, -1].values

In [35]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=72)

In [36]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [37]:
classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[129  17]
 [ 25  16]]
0.7754010695187166
